# 概要
* Vectorできれいに要約させてみる
* デフォルトだと1ノードしか見ない、複数ノード指定するとぐちゃぐちゃになる

# 参考
* だめ

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [5]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
#index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk: 

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8...
DEBUG:root:> Adding chunk: 悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き...
DEBUG:root:> Adding chunk: 

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過...
DEBUG:root:> Adding chunk: 仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と...
DEBUG:root:> Adding chunk: フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落と...
DEBUG:root:> Adding chunk: トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空に...
DEBUG:root:> Adding chunk: 悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強...
DEBUG:root:> Adding chunk: 天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人...
DEBUG:root:> Adding chunk: ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None da

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=145 request_id=90ba85813046c76de9d8fb58b3b7f463 response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4371 tokens


In [12]:
summary = index.query("詳しく要約してください", similarity_top_k=20)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u8a73\\u3057\\u304f\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8417
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=257 request_id=b93b07546022f186416048bad4c0e9c2 response_code=200
DEBUG:llama_index.indices.utils:> Top 10 nodes:
> [Node 19fb4b5c-5302-46fd-9662-32dcdbbc1a8b] [Similarity score:             0.793553] トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をも...
> [Node e6b0d726-36a2-4f13-b920-e484d94b4615] [Similari

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=20904 request_id=747144f132da769b00d952c13730cb57 response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。そして、約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。しかし、事態は想定を超えて究極の人造人間・セルが未来から出現し、セルは地球の命運を賭けた武道会「セルゲーム」を全世界に宣言した。
DEBUG:root:> Refined response: 

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。そして、約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=35214 request_id=ecb0ca640bb2e238a26f5235278a770b response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。

それから10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。しかし、事態は想定を超えて究極の人造人間・セルが未来から出現し、セルは地球の命運を賭けた武道会「セルゲーム」を全世界に宣言した。悟空はウーブと共に見果
DEBUG:roo

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=35161 request_id=6c2dcf64629e7df757067c7d05747216 response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武
DEBUG:root:> 

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=37955 request_id=b1aa92643f960efe99d6800d14153446 response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武
DEBUG:root:> 

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=33725 request_id=6004daebe1cfd8d8c03972b65b01b76f response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。

孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空
DEBUG:root:> R

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=33617 request_id=7b688dc4be81d295a0cc06a21016580c response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。

孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空
DEBUG:root:> R

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=35728 request_id=361972bc39ce7ca29d5f078961ff756f response_code=200
DEBUG:root:

トランクスという青年が悟空に未来から来たことを明かし、悟空が心臓病で命を落とすこと、3年後に2体の人造人間が現れることを告げる。悟空は特効薬で生還し、3年間修行をしてその日を迎える。その間、悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。

孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空
DEBUG:root:> R

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=35149 request_id=789531b4385ef300f754c102bc93fc21 response_code=200
DEBUG:root:

孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空は、孫悟飯が天下一武道会に出場することを知ったことをきっかけに、自らも出場するために占いババの力によって1日だけこの世に戻る。悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす。だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破。セルゲームを制した悟飯たちは、ドラゴンボールによ
DEBUG:root:> Refin

In [13]:
summary

Response(response='\n\n孫悟空少年編では、西の都からやって来た少女・ブルマと出会い、7つ集めると神龍（シェンロン）が現れるドラゴンボールの存在を知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は亀仙人の下で8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。\n\nサイヤ人編\nピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空は、孫悟飯が天下一武道会に出場することを知ったことをきっかけに、自らも出場するために占いババの力によって1日だけこの世に戻る。悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす。だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破。セルゲームを制した悟飯たちは、ドラゴンボールによ', source_nodes=[SourceNode(source_text='トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。その後本当に心臓病によって危篤状態になるも、トランクスから渡された特効薬によって生還、来るべき日に備えて3年間各々に修行してその日を迎える悟空たちであったが、事態はトランクスが知っている歴史とは大きく違うものとなり、彼さえ知らなかった人造人間たち

In [14]:
summary.source_nodes

[SourceNode(source_text='トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。その後本当に心臓病によって危篤状態になるも、トランクスから渡された特効薬によって生還、来るべき日に備えて3年間各々に修行してその日を迎える悟空たちであったが、事態はトランクスが知っている歴史とは大きく違うものとなり、彼さえ知らなかった人造人間たちまで現れ、さらに究極の人造人間・セルが未来から出現。悟空らの想定を遥かに超えた戦士が続々と現れた。\n\n人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する', doc_id='5d543477-cf78-41be-a7df-6ef201e2c31d', extra_info=None, node_info={'start': 1915, 'end': 2260}, similarity=0.7935527066891264),
 SourceNode(source_text='悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。\n\nその約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る', doc_id='5d543477-cf78-41be-a7df-6ef201e2c31d', extra_info=None, node_info={'start': 635, 'end': 948}, similarity=0.7845754457405877),
 SourceNode(